In [ ]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print("GPU available:", gpus)
else:
    print("No GPU available.")


GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Step 2: Data Preparation
!unzip -q '/content/drive/MyDrive/IMG_CLASSES.zip' -d '/content/dataset'

In [ ]:
import os
import shutil
import random

class_names = [
    "Atopic Dermatitis", "Basal Cell Carcinoma (BCC)", "Benign Keratosis-like Lesions (BKL)", "Eczema",
    "Melanocytic Nevi (NV)", "Melanoma", "Psoriasis pictures Lichen Planus and related diseases",
    "Seborrheic Keratoses and other Benign Tumors", "Tinea Ringworm Candidiasis and other Fungal Infections",
    "Warts Molluscum and other Viral Infections"
]

# Define the target number of images per class
target_num_images = 1200
# Define the base directory containing your dataset
base_dir = '/content/dataset/IMG_CLASSES'  # Update to your dataset directory

# Create a new directory to store the balanced dataset
balanced_dir = '/content/dataset/IMG_CLASSES_BALANCED'  # Update to your desired directory
if not os.path.exists(balanced_dir):
    os.makedirs(balanced_dir)

# Loop through each class folder in the original dataset
for class_name in class_names:
    class_dir = os.path.join(base_dir, class_name)
    images = os.listdir(class_dir)

    # Shuffle the images randomly
    random.shuffle(images)

    # Determine the number of images to keep for this class
    num_images_to_keep = min(len(images), target_num_images)

    # Create a new directory for this class in the balanced dataset
    balanced_class_dir = os.path.join(balanced_dir, class_name)
    if not os.path.exists(balanced_class_dir):
        os.makedirs(balanced_class_dir)

    # Copy a subset of images to the balanced dataset directory
    for image in images[:num_images_to_keep]:
        src = os.path.join(class_dir, image)
        dst = os.path.join(balanced_class_dir, image)
        shutil.copyfile(src, dst)


In [ ]:
import os

# Define the directory containing your balanced dataset
balanced_dir = '/content/dataset/IMG_CLASSES_BALANCED'  # Update to your balanced dataset directory

# Loop through each class folder in the balanced dataset
for class_name in os.listdir(balanced_dir):
    # Make sure we're only counting files in directories (class folders)
    if os.path.isdir(os.path.join(balanced_dir, class_name)):
        class_dir = os.path.join(balanced_dir, class_name)
        images = os.listdir(class_dir)
        num_images = len(images)
        print(f"{class_name}: {num_images} images")

        # Optionally, assert to make sure each class has exactly 1200 images
        assert num_images == 1200, f"{class_name} does not have 1200 images, it has {num_images} images."


Atopic Dermatitis: 1200 images
Melanocytic Nevi (NV): 1200 images
Benign Keratosis-like Lesions (BKL): 1200 images
Psoriasis pictures Lichen Planus and related diseases: 1200 images
Warts Molluscum and other Viral Infections: 1200 images
Eczema: 1200 images
Tinea Ringworm Candidiasis and other Fungal Infections: 1200 images
Seborrheic Keratoses and other Benign Tumors: 1200 images
Basal Cell Carcinoma (BCC): 1200 images
Melanoma: 1200 images


In [ ]:
from sklearn.model_selection import train_test_split
import os
import shutil

# Define class names and the number of images per class
class_names = [
    "Atopic Dermatitis", "Basal Cell Carcinoma (BCC)", "Benign Keratosis-like Lesions (BKL)", "Eczema",
    "Melanocytic Nevi (NV)", "Melanoma", "Psoriasis pictures Lichen Planus and related diseases",
    "Seborrheic Keratoses and other Benign Tumors", "Tinea Ringworm Candidiasis and other Fungal Infections",
    "Warts Molluscum and other Viral Infections"
]

class_image_counts = [1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200]

# Organize the data into training and testing sets
base_dir = '/content/dataset/IMG_CLASSES_BALANCED'  # Update to your dataset directory

# Create train and test directories
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

if not os.path.exists(train_dir):
    os.makedirs(train_dir)

if not os.path.exists(test_dir):
    os.makedirs(test_dir)

# Move images into train and test directories
for class_name, image_count in zip(class_names, class_image_counts):
    class_dir = os.path.join(base_dir, class_name)
    images = os.listdir(class_dir)

    # Split the images into training and testing sets
    train_images, test_images = train_test_split(images, test_size=0.3, random_state=42)

    # Create class-specific directories in the train and test directories
    train_class_dir = os.path.join(train_dir, class_name)
    test_class_dir = os.path.join(test_dir, class_name)

    if not os.path.exists(train_class_dir):
        os.makedirs(train_class_dir)
    if not os.path.exists(test_class_dir):
        os.makedirs(test_class_dir)

    # Move images to their respective directories
    for image in train_images:
        src = os.path.join(class_dir, image)
        dst = os.path.join(train_class_dir, image)
        shutil.move(src, dst)

    for image in test_images:
        src = os.path.join(class_dir, image)
        dst = os.path.join(test_class_dir, image)
        shutil.move(src, dst)


In [ ]:
from PIL import Image
import os

# Function to resize images in a directory
def resize_images(directory, target_size):
    for root, dirs, files in os.walk(directory):
        for file in files:
            filepath = os.path.join(root, file)
            try:
                img = Image.open(filepath)
                img = img.resize(target_size, Image.ANTIALIAS)
                img.save(filepath)
            except Exception as e:
                print(f"Failed to process {filepath}: {e}")

# Define target size for resizing
target_size = (512, 512)  # Set your desired dimensions here

# Paths to train and test directories
train_directory = '/content/dataset/IMG_CLASSES_BALANCED/train'
test_directory = '/content/dataset/IMG_CLASSES_BALANCED/test'

# Resize images in train directory
resize_images(train_directory, target_size)

# Resize images in test directory
resize_images(test_directory, target_size)


<ipython-input-10-02b14f87be96>:11: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize(target_size, Image.ANTIALIAS)


In [ ]:

!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()
from ultralytics import YOLO

from IPython.display import display, Image

Ultralytics YOLOv8.1.10 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 34.3/78.2 GB disk)


In [ ]:
import os

from ultralytics import YOLO

DATA_DIR = '/content/dataset/IMG_CLASSES_BALANCED'
# Load a model
model = YOLO("yolov8l-cls.pt")  # load a pretained model

# Use the model
results = model.train(data=DATA_DIR, epochs=50, imgsz=512)  # train the model


100%|██████████| 71.7M/71.7M [00:00<00:00, 331MB/s]


Ultralytics YOLOv8.1.10 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8l-cls.pt, data=/content/dataset/IMG_CLASSES_BALANCED, epochs=50, time=None, patience=50, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_

100%|██████████| 6.23M/6.23M [00:00<00:00, 160MB/s]


AMP: checks passed ✅


train: Scanning /content/dataset/IMG_CLASSES_BALANCED/train... 8400 images, 0 corrupt: 100%|██████████| 8400/8400 [00:02<00:00, 3080.53it/s]


...


      Epoch    GPU_mem       loss  Instances       Size


               classes   top1_acc   top5_acc: 100%|██████████| 113/113 [00:42<00:00,  2.64it/s]

                   all      0.564      0.979



      Epoch    GPU_mem       loss  Instances       Size


       4/50      5.13G      1.018         16        512:   9%|▉         | 47/525 [00:23<03:11,  2.49it/s]

In [ ]:
import cv2

# Load the image
image_path = '/content/0_29.jpg'
image = cv2.imread(image_path)

# Check if the image was successfully loaded
if image is not None:
    # Define the desired dimensions
    desired_width = 320
    desired_height = 320

    # Resize the image
    resized_image = cv2.resize(image, (desired_width, desired_height))

    # Save or use the resized image
    cv2.imwrite('/content/exi.jpg', resized_image)
else:
    print("Image could not be loaded.")


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n-cls.pt')  # load an official model
model = YOLO('/content/drive/MyDrive/YOLOALGO3Augmen/runs/classify/train/weights/best.pt')  # load a custom model

# Predict with the model
results = model('/content/Getty.jpg')  # predict on an image


image 1/1 /content/Getty.jpg: 320x320 Warts Molluscum and other Viral Infections 0.77, Psoriasis pictures Lichen Planus and related diseases 0.09, Eczema 0.09, Tinea Ringworm Candidiasis and other Fungal Infections 0.04, Atopic Dermatitis 0.00, 517.6ms
Speed: 2.5ms preprocess, 517.6ms inference, 0.1ms postprocess per image at shape (1, 3, 320, 320)


In [ ]:
import json

class_names = [
    "Atopic Dermatitis", "Basal Cell Carcinoma (BCC)", "Benign Keratosis-like Lesions (BKL)", "Eczema",
    "Melanocytic Nevi (NV)", "Melanoma", "Psoriasis pictures Lichen Planus and related diseases",
    "Seborrheic Keratoses and other Benign Tumors", "Tinea Ringworm Candidiasis and other Fungal Infections",
    "Warts Molluscum and other Viral Infections"
]

metadata = {
    "name": "Skin Disease Classification Model",
    "description": "A model for classifying skin diseases",
    "input": [{"name": "input_image", "type": "uint8", "shape": [1, 3, 320, 320]}],
    "output": [{"name": "output_scores", "type": "float32", "shape": [1, len(class_names)]}],
    "labels": class_names
}

with open('model_metadata.json', 'w') as meta_file:
    json.dump(metadata, meta_file)


In [ ]:
import cv2
import numpy as np

def preprocess_image(image_path, target_shape=(320, 320)):
    # Load the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

    # Resize the image to the target shape
    resized_image = cv2.resize(image, (target_shape[1], target_shape[0]))

    # Normalize pixel values to [0, 1] and reshape to match the required format (1, 3, 320, 320)
    input_image = resized_image.astype(np.float32) / 255.0
    input_image = np.transpose(input_image, (2, 0, 1))  # Change shape from (height, width, channels) to (channels, height, width)
    input_image = np.expand_dims(input_image, axis=0)   # Add a batch dimension

    return input_image

# Usage example:
image_path = '/content/drive/MyDrive/18_84.jpg'  # Replace with your image path
input_image = preprocess_image(image_path)
print(f"Processed image shape: {input_image.shape}")



Processed image shape: (1, 3, 320, 320)


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n-cls.pt')  # load an official model
model = YOLO('/content/drive/MyDrive/YOLOALGO3Augmen/runs/classify/train/weights/best.pt')  # load a custom model

# Export the model
model.export(format='onnx')

Ultralytics YOLOv8.0.232 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.00GHz)
YOLOv8l-cls summary (fused): 133 layers, 36197386 parameters, 0 gradients, 98.7 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/YOLOALGO3Augmen/runs/classify/train/weights/best.pt' with input shape (1, 3, 320, 320) BCHW and output shape(s) (1, 10) (69.2 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 264.0 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 9.7s, installed 1 package: ['onnx>=1.12.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success ✅ 16.7s, saved as '/content/drive/MyDrive/YOLOALGO3Augmen/runs/classify/train/weights/best.onnx' (138.1 MB)

Export complete (19.5s)
Results saved to /content/drive/MyDrive/YOLOALGO3Augmen/runs/classify/train/weights
Predict:         yolo predict

'/content/drive/MyDrive/YOLOALGO3Augmen/runs/classify/train/weights/best.onnx'

In [ ]:
import tensorflow as tf
from PIL import Image
import numpy as np

# Load the TensorFlow Lite model
model_path = '/content/drive/MyDrive/YOLOTFLITE/converted_model.tflite'  # Replace with the path to your TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output details (assuming single input and output)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load and preprocess the image
image_path = '/content/mil.jpg'  # Replace with your image path
target_width, target_height = input_details[0]['shape'][2], input_details[0]['shape'][1]

# Load the image and resize to match the model's input shape
image = Image.open(image_path)
image = image.resize((target_width, target_height))

# Convert image to numpy array, normalize pixel values to [0, 1]
image = np.array(image, dtype=np.float32) / 255.0

# Ensure image has the correct shape and format
image = np.expand_dims(image, axis=0)  # Add batch dimension

# Transpose the image if needed (depends on model's input format)
# image = np.transpose(image, (0, 3, 1, 2))  # Adjust based on model's input format

# Run inference
interpreter.set_tensor(input_details[0]['index'], image)
interpreter.invoke()

# Get the output
output = interpreter.get_tensor(output_details[0]['index'])

# Process the output as needed
# Output contains predictions or scores based on your model

# Print the output (for illustration)
print(output)


ValueError: ignored

In [ ]:
from ultralytics import YOLO
import json

# Define the path to the model and metadata file
model_path = '/content/drive/MyDrive/YOLOTF1/converted_model.tflite'
metadata_path = '/content/model_metadata.json'

# Load the model
model = YOLO(model_path, task='classify')

# Load metadata separately
with open(metadata_path, 'r') as metadata_file:
    metadata = json.load(metadata_file)

# Now you can use the metadata along with the model
# For example, accessing class names from metadata
class_names = metadata['labels']

# Make predictions with the model
results = model('/content/mil.jpg')  # Replace 'mil.jpg' with your image path


Loading /content/drive/MyDrive/YOLOTF1/converted_model.tflite for TensorFlow Lite inference...
WARNING ⚠️ Metadata not found for 'model=/content/drive/MyDrive/YOLOTF1/converted_model.tflite'



ValueError: ignored

In [ ]:
!pip install onnx-tf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 35.1 MB/s eta 0:00:00


In [ ]:
import onnx
import tensorflow as tf
from onnx_tf.backend import prepare

# Load ONNX model
onnx_model = onnx.load('/content/drive/MyDrive/YOLOALGO3Augmen/runs/classify/train/weights/best.onnx')

# Prepare ONNX model for TensorFlow
tf_rep = prepare(onnx_model)
tf_rep.export_graph('/content/drive/MyDrive/YOLOALGO3Augmen/runs/classify/train/weights/yolov8.pb')

# Convert TensorFlow model (.pb) to TensorFlow Lite with TF Select enabled
converter = tf.lite.TFLiteConverter.from_saved_model('/content/drive/MyDrive/YOLOALGO3Augmen/runs/classify/train/weights/yolov8.pb')
converter.experimental_new_converter = True  # Enable the new converter
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TensorFlow Lite built-in ops
    tf.lite.OpsSet.SELECT_TF_OPS  # Enable TF Select for handling unsupported ops
]
tflite_model = converter.convert()

# Save TensorFlow Lite model
tflite_model_file = '/content/drive/MyDrive/YOLOALGO3Augmen/runs/classify/train/weights/yolov8.tflite'
with open(tflite_model_file, 'wb') as f:
    f.write(tflite_model)


In [ ]:
import numpy as np
import tensorflow as tf
from PIL import Image

# Load the TensorFlow Lite model
tflite_model_file = '/content/drive/MyDrive/YOLOTF1/converted_model.tflite'
interpreter = tf.lite.Interpreter(model_path=tflite_model_file)
interpreter.allocate_tensors()

# Get input and output details (assuming single input and output)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load and preprocess the image
image_path = '/content/drive/MyDrive/mm.jpg'  # Replace with your image path
input_shape = (320, 320)  # Model's input shape

# Load the image and resize to match the model's input shape
image = Image.open(image_path)
image = image.convert("RGB")  # Ensure RGB format
image = image.resize(input_shape)

# Normalize pixel values to [0, 1] and convert to numpy array
image = np.array(image, dtype=np.float32) / 255.0

# Rearrange dimensions to match the model's expected input shape [1, 3, 320, 320]
input_data = np.moveaxis(image, -1, 0)  # Move channel axis to front
input_data = np.expand_dims(input_data, axis=0)  # Add batch dimension

# Check if the input shape matches the model's expected input shape
if tuple(input_details[0]['shape']) != input_data.shape:
    raise ValueError(
        f"Input shape mismatch, expected {input_details[0]['shape']} but got {input_data.shape}"
    )

# Set the input tensor
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run inference
interpreter.invoke()

# Get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])
predicted_class_index = np.argmax(output_data)  # Get index of highest probability class
predicted_class_name = class_names[predicted_class_index]  # Get the class name

# Display the predicted class name
print(f"The predicted class is: {predicted_class_name}")

The predicted class is: Tinea Ringworm Candidiasis and other Fungal Infections


In [ ]:
import zipfile

def compress_model(input_file_name, output_file_name):
    with zipfile.ZipFile(output_file_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(input_file_name, arcname=input_file_name)

# Specify the name of your existing .tflite file
input_file_name = '/content/drive/MyDrive/YOLOALGO3Augmen/runs/classify/train/weights/yolov8.tflite'
# Specify the desired output zip file name
output_file_name = '/content/drive/MyDrive/YOLOALGO3Augmen/runs/classify/train/weights/model_compressed.zip'

compress_model(input_file_name, output_file_name)


In [ ]:
def decompress_model(input_zip_file, extract_folder='.'):
    with zipfile.ZipFile(input_zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)

# Specify the name of your compressed file and the extraction folder
input_zip_file = 'model_compressed.zip'
extract_folder = '.'  # Current directory

decompress_model(input_zip_file, extract_folder)
